In [1]:
# ✂️ AUTOMATED SEGMENTATION → Start at Trigger Onset
# Output: derivatives/crop/On/<task>/ and crop/Off/<task>/
# Filename: sub6_eyes_closed_on.fif

# %%
import os
import mne
import numpy as np
from tqdm import tqdm

# --- CONFIGURATION ---
BASE_DERIVATIVES_DIR = "/home/jaizor/jaizor/xtra/derivatives/eeg"
OUTPUT_BASE_DIR = "/home/jaizor/jaizor/xtra/derivatives/crop"  # ← Now 'crop' instead of 'source_ready'

TASK = "rest_hand_foot"
SESSIONS = ["DBSOFF", "DBSON"]
TRIGGERS = ['c', 'o', '1a']
TASK_NAMES = ['eyes_closed', 'eyes_open', 'hands_move']  # lowercase
DURATION = 60.0     # 60 seconds
OUTPUT_SUFFIX = ".fif"

# Map session to folder and filename suffix
SESSION_CONFIG = {
    "DBSOFF": {"folder": "Off", "suffix": "_off"},
    "DBSON":  {"folder": "On",  "suffix": "_on"}
}

# --- GET ALL SUBJECTS ---
def get_all_subjects(base_dir: str) -> list:
    subjects = []
    for item in os.listdir(base_dir):
        if item.startswith('sub-'):
            subjects.append(item)
    return sorted(subjects, key=lambda x: int(x.split('-')[1]))

subjects = get_all_subjects(BASE_DERIVATIVES_DIR)
print(f"✂️ Found {len(subjects)} subjects: {subjects}")
print("="*80)

# --- DETECT FIRST TRIGGER ONSET ---
def detect_first_trigger_onsets(raw: mne.io.Raw, triggers: list) -> dict:
    onsets = {}
    sfreq = raw.info['sfreq']

    for trig in triggers:
        if trig not in raw.ch_names:
            print(f"    ❌ Trigger '{trig}' not found in channels!")
            onsets[trig] = None
            continue

        data = raw.get_data(picks=[trig])[0]
        diff = np.diff((data > 0.5).astype(int))  # Rising edge detection
        onset_indices = np.where(diff == 1)[0] + 1
        onset_times = onset_indices / sfreq

        if len(onset_times) == 0:
            print(f"    ⚠️  No rising edges for '{trig}'!")
            onsets[trig] = None
        else:
            onsets[trig] = onset_times[0]

    return onsets

# --- MAIN PROCESSING LOOP ---
success_count = 0
failure_count = 0

for subject in tqdm(subjects, desc="Processing Subjects", unit="subject"):
    for session in SESSIONS:
        # Build input file path
        task_session_dir = os.path.join(BASE_DERIVATIVES_DIR, subject, f"{TASK}_{session}")
        filename = f"{subject}_ses-{session}_task-{TASK}_eeg_ica_cleaned_raw.fif"
        filepath = os.path.join(task_session_dir, filename)

        if not os.path.exists(filepath):
            tqdm.write(f"⚠️  File not found: {filename} ({subject}/{session})")
            continue

        try:
            # Load data
            raw = mne.io.read_raw_fif(filepath, preload=True)
            total_duration = raw.times[-1]

            # Detect trigger onsets
            onsets = detect_first_trigger_onsets(raw, TRIGGERS)

            # Process each trigger/task
            for trig, task_name in zip(TRIGGERS, TASK_NAMES):
                onset = onsets.get(trig)
                if onset is None:
                    tqdm.write(f"❌ Skipping {task_name} for {subject}/{session}: trigger '{trig}' not found or no onset")
                    continue

                tmin = onset
                tmax = tmin + DURATION

                # Validate time bounds
                if tmin >= total_duration:
                    tqdm.write(f"❌ {task_name} for {subject}/{session}: start {tmin:.3f}s > duration {total_duration:.3f}s")
                    continue
                if tmin < 0:
                    tqdm.write(f"❌ {task_name} for {subject}/{session}: invalid start {tmin:.3f}s")
                    continue
                if tmax > total_duration:
                    tqdm.write(f"⚠️  {task_name} for {subject}/{session}: truncating to {total_duration:.3f}s")
                    tmax = total_duration
                if tmax <= tmin:
                    tqdm.write(f"❌ {task_name} for {subject}/{session}: no valid data segment")
                    continue

                # Crop segment
                raw_crop = raw.copy().crop(tmin=tmin, tmax=tmax)

                # Get session config
                cfg = SESSION_CONFIG[session]
                on_off_folder = cfg["folder"]  # "On" or "Off"
                file_suffix = cfg["suffix"]   # "_on" or "_off"

                # Create path: crop/On/eyes_closed/
                output_task_dir = os.path.join(OUTPUT_BASE_DIR, on_off_folder, task_name)
                os.makedirs(output_task_dir, exist_ok=True)

                # Simplify subject name: sub-06 → sub6
                simple_sub = 'sub' + subject.split('-')[1].lstrip('0')

                # Filename includes _on or _off
                output_filename = f"{simple_sub}_{task_name}{file_suffix}{OUTPUT_SUFFIX}"
                output_path = os.path.join(output_task_dir, output_filename)

                # Save
                raw_crop.save(output_path, overwrite=True)
                tqdm.write(f"✅ Saved: {output_filename} → {output_task_dir} ({tmax-tmin:.1f}s) [{subject}/{session}]")
                success_count += 1

        except Exception as e:
            tqdm.write(f"❌ FAILED: {subject}/{session} — {e}")
            failure_count += 1

# --- FINAL SUMMARY ---
print(f"\n" + "="*80)
print("🎉 SEGMENTATION COMPLETE")
print("="*80)
print(f"   Successfully saved segments: {success_count}")
print(f"   Failed or skipped:           {failure_count}")
print(f"   Output structure: {OUTPUT_BASE_DIR}/<On|Off>/<task_name>/")

✂️ Found 12 subjects: ['sub-01', 'sub-02', 'sub-03', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09', 'sub-10', 'sub-11', 'sub-12', 'sub-14']


Processing Subjects:   0%|          | 0/12 [00:00<?, ?subject/s]

Opening raw data file /home/jaizor/jaizor/xtra/derivatives/eeg/sub-01/rest_hand_foot_DBSOFF/sub-01_ses-DBSOFF_task-rest_hand_foot_eeg_ica_cleaned_raw.fif...
    Range : 65419 ... 345902 =    130.838 ...   691.804 secs
Ready.
Reading 0 ... 280483  =      0.000 ...   560.966 secs...
Writing /home/jaizor/jaizor/xtra/derivatives/crop/Off/eyes_closed/sub1_eyes_closed_off.fif
Closing /home/jaizor/jaizor/xtra/derivatives/crop/Off/eyes_closed/sub1_eyes_closed_off.fif
[done]


/tmp/ipykernel_124489/1735541731.py:131: RuntimeWarning: This filename (/home/jaizor/jaizor/xtra/derivatives/crop/Off/eyes_closed/sub1_eyes_closed_off.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_crop.save(output_path, overwrite=True)
Processing Subjects:   0%|          | 0/12 [00:00<?, ?subject/s]

✅ Saved: sub1_eyes_closed_off.fif → /home/jaizor/jaizor/xtra/derivatives/crop/Off/eyes_closed (60.0s) [sub-01/DBSOFF]
Writing /home/jaizor/jaizor/xtra/derivatives/crop/Off/eyes_open/sub1_eyes_open_off.fif


...

✅ Saved: sub14_eyes_open_off.fif → /home/jaizor/jaizor/xtra/derivatives/crop/Off/eyes_open (60.0s) [sub-14/DBSOFF]
Writing /home/jaizor/jaizor/xtra/derivatives/crop/Off/hands_move/sub14_hands_move_off.fif


/tmp/ipykernel_124489/1735541731.py:131: RuntimeWarning: This filename (/home/jaizor/jaizor/xtra/derivatives/crop/Off/hands_move/sub14_hands_move_off.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_crop.save(output_path, overwrite=True)


Closing /home/jaizor/jaizor/xtra/derivatives/crop/Off/hands_move/sub14_hands_move_off.fif
[done]


Processing Subjects: 100%|██████████| 12/12 [00:21<00:00,  1.80s/subject]

✅ Saved: sub14_hands_move_off.fif → /home/jaizor/jaizor/xtra/derivatives/crop/Off/hands_move (60.0s) [sub-14/DBSOFF]
⚠️  File not found: sub-14_ses-DBSON_task-rest_hand_foot_eeg_ica_cleaned_raw.fif (sub-14/DBSON)

🎉 SEGMENTATION COMPLETE
   Successfully saved segments: 63
   Failed or skipped:           0
   Output structure: /home/jaizor/jaizor/xtra/derivatives/crop/<On|Off>/<task_name>/


In [2]:
# ✂️ BIMA SEGMENTATION (DBSOFF ONLY) → Start at '1a' Onset
# Output: derivatives/crop/Off/bima_activity/subX_bima_activity_off.fif

# %%
import os
import mne
import numpy as np
from tqdm import tqdm

# --- CONFIGURATION ---
BASE_DERIVATIVES_DIR = "/home/jaizor/jaizor/xtra/derivatives/eeg"
OUTPUT_BASE_DIR = "/home/jaizor/jaizor/xtra/derivatives/crop"  # ← Unified with other script

TASK = "bima"
SESSION = "DBSOFF"          # Only process DBSOFF
TRIGGER = '1a'
TASK_NAME = 'bima_activity' # lowercase
DURATION = 60.0
OUTPUT_SUFFIX = ".fif"

# Session → folder and suffix mapping (even though only one session)
SESSION_CONFIG = {
    "DBSOFF": {"folder": "Off", "suffix": "_off"}
}

# --- GET ALL SUBJECTS ---
def get_all_subjects(base_dir: str) -> list:
    subjects = []
    for item in os.listdir(base_dir):
        if item.startswith('sub-'):
            subjects.append(item)
    return sorted(subjects, key=lambda x: int(x.split('-')[1]))

subjects = get_all_subjects(BASE_DERIVATIVES_DIR)
print(f"✂️ Found {len(subjects)} subjects: {subjects}")
print("="*80)

# --- DETECT FIRST '1a' ONSET ---
def detect_first_1a_onset(raw: mne.io.Raw) -> float:
    if TRIGGER not in raw.ch_names:
        print(f"    ❌ Trigger '{TRIGGER}' not found in channels!")
        return None

    data = raw.get_data(picks=[TRIGGER])[0]
    diff = np.diff((data > 0.5).astype(int))  # Rising edge: detect 0→1
    onset_indices = np.where(diff == 1)[0] + 1
    sfreq = raw.info['sfreq']
    onset_times = onset_indices / sfreq

    if len(onset_times) == 0:
        print(f"    ⚠️  No rising edges detected for '{TRIGGER}'!")
        return None

    return onset_times[0]  # First trigger onset

# --- MAIN PROCESSING LOOP ---
success_count = 0
failure_count = 0

for subject in tqdm(subjects, desc="Processing BIMA (DBSOFF)", unit="subject"):
    # Build file path
    task_session_dir = os.path.join(BASE_DERIVATIVES_DIR, subject, f"{TASK}_{SESSION}")
    filename = f"{subject}_ses-{SESSION}_task-{TASK}_eeg_ica_cleaned_raw.fif"
    filepath = os.path.join(task_session_dir, filename)

    if not os.path.exists(filepath):
        tqdm.write(f"⚠️  File not found: {filename} ({subject})")
        failure_count += 1
        continue

    try:
        # Load data
        raw = mne.io.read_raw_fif(filepath, preload=True)
        total_duration = raw.times[-1]

        # Detect first '1a' trigger onset
        onset = detect_first_1a_onset(raw)
        if onset is None:
            tqdm.write(f"❌ Skipping {subject}: could not detect '{TRIGGER}' onset")
            failure_count += 1
            continue

        tmin = onset
        tmax = tmin + DURATION

        # Validate time bounds
        if tmin >= total_duration:
            tqdm.write(f"❌ {subject}: start {tmin:.3f}s > total duration {total_duration:.3f}s")
            failure_count += 1
            continue
        if tmin < 0:
            tqdm.write(f"❌ {subject}: invalid start time {tmin:.3f}s")
            failure_count += 1
            continue
        if tmax > total_duration:
            tqdm.write(f"⚠️  {subject}: truncating segment from {tmax:.3f}s to {total_duration:.3f}s")
            tmax = total_duration
        if tmax <= tmin:
            tqdm.write(f"❌ {subject}: empty time segment ({tmin:.3f}s to {tmax:.3f}s)")
            failure_count += 1
            continue

        # Crop segment
        raw_crop = raw.copy().crop(tmin=tmin, tmax=tmax)

        # Get output config
        cfg = SESSION_CONFIG[SESSION]
        on_off_folder = cfg["folder"]   # "Off"
        file_suffix = cfg["suffix"]     # "_off"

        # Create output directory: crop/Off/bima_activity/
        output_task_dir = os.path.join(OUTPUT_BASE_DIR, on_off_folder, TASK_NAME)
        os.makedirs(output_task_dir, exist_ok=True)

        # Simplify subject name: sub-06 → sub6
        simple_sub = 'sub' + subject.split('-')[1].lstrip('0')

        # Filename includes _off
        output_filename = f"{simple_sub}_{TASK_NAME}{file_suffix}{OUTPUT_SUFFIX}"
        output_path = os.path.join(output_task_dir, output_filename)

        # Save cropped data
        raw_crop.save(output_path, overwrite=True)
        tqdm.write(f"✅ Saved: {output_filename} → {output_task_dir} ({tmax - tmin:.1f}s) [{subject}]")
        success_count += 1

    except Exception as e:
        tqdm.write(f"❌ FAILED: {subject} — {e}")
        failure_count += 1

# --- FINAL SUMMARY ---
print(f"\n" + "="*80)
print("🎉 BIMA SEGMENTATION COMPLETE")
print("="*80)
print(f"   Successfully saved segments: {success_count}")
print(f"   Failed or skipped:           {failure_count}")
print(f"   Output structure: {OUTPUT_BASE_DIR}/Off/{TASK_NAME}/")

✂️ Found 12 subjects: ['sub-01', 'sub-02', 'sub-03', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09', 'sub-10', 'sub-11', 'sub-12', 'sub-14']


Processing BIMA (DBSOFF):   0%|          | 0/12 [00:00<?, ?subject/s]

Opening raw data file /home/jaizor/jaizor/xtra/derivatives/eeg/sub-01/bima_DBSOFF/sub-01_ses-DBSOFF_task-bima_eeg_ica_cleaned_raw.fif...
    Range : 125209 ... 340681 =    250.418 ...   681.362 secs
Ready.
Reading 0 ... 215472  =      0.000 ...   430.944 secs...
Writing /home/jaizor/jaizor/xtra/derivatives/crop/Off/bima_activity/sub1_bima_activity_off.fif
Closing /home/jaizor/jaizor/xtra/derivatives/crop/Off/bima_activity/sub1_bima_activity_off.fif
[done]


/tmp/ipykernel_124489/1756197261.py:123: RuntimeWarning: This filename (/home/jaizor/jaizor/xtra/derivatives/crop/Off/bima_activity/sub1_bima_activity_off.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_crop.save(output_path, overwrite=True)
Processing BIMA (DBSOFF):   8%|▊         | 1/12 [00:00<00:10,  1.04subject/s]

✅ Saved: sub1_bima_activity_off.fif → /home/jaizor/jaizor/xtra/derivatives/crop/Off/bima_activity (60.0s) [sub-01]
Opening raw data file /home/jaizor/jaizor/xtra/derivatives/eeg/sub-02/bima_DBSOFF/sub-02_ses-DBSOFF_task-bima_eeg_ica_cleaned_raw.fif...
    Range : 57645 ... 306110 =    115.290 ...   612.220 secs
Ready.
Reading 0 ... 248465  =      0.000 ...   496.930 secs...
Writing /home/jaizor/jaizor/xtra/derivatives/crop/Off/bima_activity/sub2_bima_activity_off.fif
Closing /home/jaizor/jaizor/xtra/derivatives/crop/Off/bima_activity/sub2_bima_activity_off.fif
[done]


...

✅ Saved: sub11_bima_activity_off.fif → /home/jaizor/jaizor/xtra/derivatives/crop/Off/bima_activity (60.0s) [sub-11]
Opening raw data file /home/jaizor/jaizor/xtra/derivatives/eeg/sub-12/bima_DBSOFF/sub-12_ses-DBSOFF_task-bima_eeg_ica_cleaned_raw.fif...
    Range : 32867 ... 243522 =     65.734 ...   487.044 secs
Ready.
Reading 0 ... 210655  =      0.000 ...   421.310 secs...
Writing /home/jaizor/jaizor/xtra/derivatives/crop/Off/bima_activity/sub12_bima_activity_off.fif
Closing /home/jaizor/jaizor/xtra/derivatives/crop/Off/bima_activity/sub12_bima_activity_off.fif
[done]


/tmp/ipykernel_124489/1756197261.py:123: RuntimeWarning: This filename (/home/jaizor/jaizor/xtra/derivatives/crop/Off/bima_activity/sub12_bima_activity_off.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_crop.save(output_path, overwrite=True)
Processing BIMA (DBSOFF):  92%|█████████▏| 11/12 [00:08<00:00,  1.61subject/s]

✅ Saved: sub12_bima_activity_off.fif → /home/jaizor/jaizor/xtra/derivatives/crop/Off/bima_activity (60.0s) [sub-12]
Opening raw data file /home/jaizor/jaizor/xtra/derivatives/eeg/sub-14/bima_DBSOFF/sub-14_ses-DBSOFF_task-bima_eeg_ica_cleaned_raw.fif...
    Range : 61063 ... 324418 =    122.126 ...   648.836 secs
Ready.
Reading 0 ... 263355  =      0.000 ...   526.710 secs...
Writing /home/jaizor/jaizor/xtra/derivatives/crop/Off/bima_activity/sub14_bima_activity_off.fif
Closing /home/jaizor/jaizor/xtra/derivatives/crop/Off/bima_activity/sub14_bima_activity_off.fif
[done]


/tmp/ipykernel_124489/1756197261.py:123: RuntimeWarning: This filename (/home/jaizor/jaizor/xtra/derivatives/crop/Off/bima_activity/sub14_bima_activity_off.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_crop.save(output_path, overwrite=True)
Processing BIMA (DBSOFF): 100%|██████████| 12/12 [00:09<00:00,  1.33subject/s]

✅ Saved: sub14_bima_activity_off.fif → /home/jaizor/jaizor/xtra/derivatives/crop/Off/bima_activity (60.0s) [sub-14]

🎉 BIMA SEGMENTATION COMPLETE
   Successfully saved segments: 12
   Failed or skipped:           0
   Output structure: /home/jaizor/jaizor/xtra/derivatives/crop/Off/bima_activity/
